In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
from Bio import SeqUtils
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

In [1]:
def rfimp(feats,pnr):
    
    r = res[feats].copy()
    pnr = pnr.drop(columns=['Id', 'MolTyp', 'Topol', 'Des'])

    ps = pnr.loc[pnr['Gen'] == 1]
    ns = pnr.loc[pnr['Gen'] == 0]

    pnrs = pd.concat([ps,ns])
    
    #Splitting features and labels
    X = pnrs.iloc[:, 3:].values.astype(float)
    y = pnrs.iloc[:, 1].values.astype(float)
    
    # Splitting the dataset into the Training set and Test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 15, criterion = 'entropy')
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    
    cv_bin = cross_val_score(classifier, X, y, cv=5, scoring='f1')
    cv_mac = cross_val_score(classifier, X, y, cv=5, scoring='f1_macro')
    f1bin = f1_score(y_test, y_pred, average='binary')
    f1mac = f1_score(y_test, y_pred, average='macro')
    return cv_bin, cv_mac, f1bin, f1mac

In [8]:
aclist = []

for i in range(20):
    fmac = whole(pnr)
    aclist.append(fmac.mean())
    print(fmac.mean(), fmac.std() *2)

print(np.mean(aclist))

0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653658 0.07213781802967897
0.9201008909653657
